In [ ]:
import torch

In [ ]:
print(torch.__version__)

2.0.0+cu118


In [ ]:
!pip install pyg_lib torch_scatter torch_sparse -f https://data.pyg.org/whl/torch-2.0.0+cu118.html
!pip install torch-geometric
!pip install torch-geometric-temporal


In [ ]:
from torch_geometric.nn import GCNConv
import sys
import torch_geometric
from torch_geometric_temporal.nn.recurrent.temporalgcn import TGCN
from torch_geometric_temporal.nn.recurrent.temporalgcn import TGCN2
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TGCNMODEL(nn.Module):
    def __init__(self, in_channels, out_channels,hidden_channel,emb, num_nodes, num_layers, dropout):
        super(TGCNMODEL, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.hidden_channel=hidden_channel
        self.num_nodes = num_nodes
        self.num_layers = num_layers
        self.dropout = dropout

        # define the temporal convolutional layers
        self.temp_conv_layers = nn.ModuleList()
        self.temp_conv_layers.append(nn.Conv2d(in_channels, hidden_channel, kernel_size=(1, 3)))
        self.temp_conv_layers.append(nn.Conv2d(hidden_channel, out_channels, kernel_size=(1, 3)))

        # define the attention mechanism
        self.attention = nn.MultiheadAttention(emb, num_heads=2, dropout=dropout,batch_first=True)

        # define the dropout layer
        self.dropout_layer = nn.Dropout(dropout)

        # define the output layer
        self.output_layer = nn.Linear(emb, 1)


    def forward(self, x, adj_matrices):
        x=x.permute(0,3,1,2)
        # x has shape (batch_size, seq_length, num_nodes, in_channels)
        # adj_matrices has shape (batch_size, num_nodes, num_nodes)


        # apply the temporal convolutional layers
        h = x
        for i in range(self.num_layers):
            # apply the convolution
            h = self.temp_conv_layers[i](h)

            # apply the adjacency matrix
            batch_size = h.size(0)
            adj_matrices_i = adj_matrices.unsqueeze(dim=1).repeat(1, h.size(1), 1, 1)
            h=torch.einsum('bsnj,bsjc->bsnc', (adj_matrices_i,h))
            # h = torch.einsum('bsnc,bnc->bsnl', (adj_matrices_i,h))

            # apply the activation function
            h = F.relu(h)

            # apply dropout
            h = self.dropout_layer(h)

        # compute the self-attention
        h = h.permute(0, 2, 1, 3)# shape (batch_size, seq_length, num_nodes, out_channels), batch_size-node-sequence-feature
        dim3=h.shape[2]
        dim4=h.shape[3]
        dim1=h.shape[0]
        h = h.reshape(-1, dim3, dim4) # shape (batch_size * num_nodes,seq_length, out_channels)
        h = self.attention (h, h, h)[0]
        h = h.reshape(dim1, self.num_nodes, dim3, dim4).permute(0, 2, 1, 3)

        # compute the output
        h = h.mean(dim=1) # take the mean over the sequence dimension
        # h = self.output_layer(h).view(32, -1, 26)
        h = self.output_layer(h).squeeze(dim=-1) #.squeeze(dim=-2) # shape (batch_size, num_nodes)

        return h
